# INFO2950 Project - Phase V

## Table of Contents
1. Introduction <br>
a. Background <br>
b. Research Questions 
2. Data Description
3. Data Cleaning
4. Preregistration Statement
5. Data Analysis
6. Evaluation of Significance
7. Interpretation and Conclusions
8. Limitations
9. Sources

## 1. Introduction

### 1a. Background

### 1b. Research Questions

Our overarching question we are looking to answer is if there is a relationship in the United States between grocery stores, fast food restaurants, and the demographics in each county? This lead us to ask the following subquestions:
- Is there a relationship between food availability, both grocery stores and restaurants, and county demographics across the United States? 
- Are there specific kinds of grocery stores and restaurants depending on the demographics in the county? 
- Is there a relationship between grocery stores and fast food restaurants within each county?

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import duckdb
from sklearn.linear_model import LinearRegression, LogisticRegression

## 2. Data Description

## 3. Data Cleaning

## 4. Preregistration Statement

## 5. Data Analysis

## 6. Evaluation of Significance

## 7. Interpretation and Conclusions

## 8. Limitations

## 9. Sources